In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.warp import reproject, calculate_default_transform, Resampling
from rasterio.features import shapes
from rasterio.features import rasterize
from rasterio.transform import from_origin
import rasterio.mask
from rasterio.io import MemoryFile
from shapely.geometry import shape
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Patch
from matplotlib.lines import Line2D


In [ ]:


uk_boundary_27700_path = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/uk_boundary_27700.geojson"

dem_tif_path = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/uk_dem_clipped_UK_EPSG27700.tif"
nc_path = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/wind_speed.nc"

capacity_tif_27700 = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/GBR_capacity-factor_IEC1_EPSG27700.tif"
powerdensity_tif_27700 = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/GBR_power-density_100m_EPSG27700.tif"

lines_path_27700 = "/Users/lct/Desktop/0093/cw2/2nd_Assignment-20251218 2/datasets/uk_transmission_lines_EPSG27700.geojson"

thr_wind = 4.5
thr_cf = 0.30
thr_pd = 400.0
thr_slope = 15.0
thr_alt = 600.0

uk = gpd.read_file(uk_boundary_27700_path)
uk["geometry"] = uk.geometry.make_valid()
uk = uk[uk.geometry.notna() & ~uk.geometry.is_empty]
uk_union = uk.unary_union

with rasterio.open(dem_tif_path) as src:
    dem_arr, tr = rasterio.mask.mask(src, [uk_union], crop=True, filled=True, nodata=np.nan)
    dem = dem_arr[0].astype("float32")
    if src.nodata is not None:
        dem = np.where(dem == src.nodata, np.nan, dem)
    ref_crs = src.crs

h, w = dem.shape
left, top = tr * (0, 0)
right, bottom = tr * (w, h)
extent = (left, right, bottom, top)

dx = abs(tr.a)
dy = abs(tr.e)
dz_dy, dz_dx = np.gradient(dem, dy, dx)
slope_pct = np.sqrt(dz_dx**2 + dz_dy**2) * 100.0

slope_ok = np.isfinite(slope_pct) & (slope_pct <= thr_slope)
alt_ok = np.isfinite(dem) & (dem <= thr_alt)

x_centers = left + dx * (np.arange(w) + 0.5)
y_centers = top - dy * (np.arange(h) + 0.5)

ds = xr.open_dataset(nc_path)
da = ds["wind_speed"].mean(dim="season", skipna=True)
if not (("x" in da.coords) and ("y" in da.coords)):
    raise ValueError("wind_speed.nc must have x and y coordinates.")
wind_on_ref = da.interp(x=x_centers, y=y_centers, method="nearest").values
wind_ok = np.isfinite(wind_on_ref) & (wind_on_ref > thr_wind)

def sample_raster_on_ref(tif_path):
    with rasterio.open(tif_path) as src:
        if src.crs is not None and ref_crs is not None and str(src.crs) != str(ref_crs):
            raise ValueError(f"Raster CRS mismatch. Need EPSG:27700 version: {tif_path}")
        coords = [(float(x), float(y)) for y in y_centers for x in x_centers]
        vals = np.fromiter((v[0] for v in src.sample(coords)), dtype="float32", count=len(coords))
        vals = vals.reshape((h, w))
        if src.nodata is not None:
            vals = np.where(vals == src.nodata, np.nan, vals)
        return vals

cf = sample_raster_on_ref(capacity_tif_27700)
pd = sample_raster_on_ref(powerdensity_tif_27700)

cf_ok = np.isfinite(cf) & (cf >= thr_cf)
pd_ok = np.isfinite(pd) & (pd > thr_pd)

lines = gpd.read_file(lines_path_27700)
lines["geometry"] = lines.geometry.make_valid()
lines = lines[lines.geometry.notna() & ~lines.geometry.is_empty]
lines = gpd.clip(lines, uk)

line_mask = rasterize(
    [(geom, 1) for geom in lines.geometry],
    out_shape=(h, w),
    transform=tr,
    fill=0,
    dtype="uint8"
).astype(bool)

valid = np.isfinite(dem)
suitable = valid & wind_ok & cf_ok & pd_ok & slope_ok & alt_ok & (~line_mask)

final = np.where(valid, suitable.astype(np.uint8), np.nan)
final = np.ma.masked_invalid(final)

cmap = ListedColormap(["purple", "green"])
cmap.set_bad(alpha=0)

fig, ax = plt.subplots(figsize=(9, 11), dpi=600)
ax.imshow(final, extent=extent, origin="upper", cmap=cmap, vmin=0, vmax=1)

uk.boundary.plot(ax=ax, color="black", linewidth=0.3)

ax.legend(
    handles=[
        Patch(facecolor="green", edgecolor="none", label="Suitable (W1–W6)"),
        Patch(facecolor="purple", edgecolor="none", label="Unsuitable (fails any)"),
    ],
    loc="upper left",
    frameon=True
)

ax.set_axis_off()
plt.tight_layout()
plt.show()